# crypto token correllation Heatmap

In [3]:
import os
from dotenv import load_dotenv
import requests
import pandas as pd

load_dotenv()  # Load environment variables from .env

api_key = os.getenv("GECKO_API")  # Get API key from .env
coin_id = "bitcoin"
vs_currency = "usd"
days = "30"

url = f"https://pro-api.coingecko.com/api/v3/coins/{coin_id}/market_chart"
params = {
    "vs_currency": vs_currency,
    "days": days
}
headers = {
    "accept": "application/json",
    "x-cg-pro-api-key": api_key
}

response = requests.get(url, headers=headers, params=params)
print(response.text)


{"status":{"timestamp":"2025-08-21T10:10:03.973+00:00","error_code":10002,"error_message":"API Key Missing. Please make sure you're using the right authentication method.  For Pro API, please visit: https://docs.coingecko.com/reference/authentication.  For Public/Demo API, please visit: https://docs.coingecko.com/v3.0.1/reference/introduction"}}


In [1]:
print(api_key)


NameError: name 'api_key' is not defined

In [ ]:
data = response.json()

# Convert 'prices' to DataFrame
df = pd.DataFrame(data['prices'], columns=['timestamp', 'price'])
df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')

print(df.head())

# Create Function

In [ ]:
import os
from dotenv import load_dotenv
import requests
import pandas as pd
import joblib


load_dotenv()  # Load environment variables from .env

api_key = os.getenv("GECKO_API")  # Get API key from .env
coin_id = "bitcoin"
vs_currency = "usd"
days = "30"



def fetch_token_price_history(token_ids, vs_currency="usd", days="30"):
    """
    Fetch historical price data for multiple tokens from CoinGecko Pro API.
    
    Args:
        token_ids (list): List of token ids (e.g., ['bitcoin', 'ethereum']).
        vs_currency (str): The target currency (default 'usd').
        days (str): Number of days to fetch (default '30').
        
    Returns:
        pd.DataFrame: DataFrame with timestamps as index and tokens as columns.
    """
    
    load_dotenv()
    api_key = os.getenv("GECKO_API")
    price_dfs = []

    for coin_id in token_ids:
        url = f"https://pro-api.coingecko.com/api/v3/coins/{coin_id}/market_chart"
        params = {
            "vs_currency": vs_currency,
            "days": days
        }
        headers = {
            "accept": "application/json",
            "x-cg-pro-api-key": api_key
        }
        response = requests.get(url, headers=headers, params=params)
        data = response.json()
        df = pd.DataFrame(data['prices'], columns=['timestamp', coin_id])
        df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')
        df.set_index('timestamp', inplace=True)
        df = df.resample('1h').last()  # or '1D' for daily
        price_dfs.append(df)
        

    # Merge all dataframes on timestamp
    result_df = pd.concat(price_dfs, axis=1)
    return result_df
    

ModuleNotFoundError: No module named 'joblib'

In [ ]:
# Example: Get tokens from user input
tokens = input("Enter token IDs separated by commas (e.g., bitcoin,ethereum,solana): ").split(",")
tokens = [t.strip() for t in tokens]

df_prices = fetch_token_price_history(tokens, days="30")
print(df_prices.head())

# Save with joblib if needed
save_dir = "/Users/olaoluwatunmise/token-heatmap/Database"
os.makedirs(save_dir, exist_ok=True)
joblib.dump(df_prices, os.path.join(save_dir, "df_prices.joblib"))